Auteurs : Groupe 8
- Arouk Amine
- Azougagh Fouad
- Benhamouche Sofiane
- Dahmi Issam
- Hadda Hassan

Clients : 
- Mme. Bahsoun Wahiba
- M. Mokadem Riad



In [ ]:
from pandas import read_csv
import numpy as np 
import pandas as pd
from sqlalchemy import create_engine
import cx_Oracle
import datetime
from tqdm import tqdm
import re
import random

In [ ]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install  cx_Oracle

Note: you may need to restart the kernel to use updated packages.


In [2]:
df=pd.read_csv('C:/Users/Inspiron/Desktop/Dashboard_V7.csv')
df

In [ ]:
#calcule le nombre d'occurences dans abstract 
compte = 0
for i in range(0,len(df)) :
    if ('multiclass classification' in str(df['Abstract'][i]).lower()) :
        compte+=1
print(compte)

8


In [ ]:
######### table Article #############
connection = engine.raw_connection()
cursor = connection.cursor()

for i in range(0,100) :
    identifiant = df['id_article'][i]
    titre = df['Titre'][i]
    annee = df['Annee'][i]
    cursor.execute("insert into Article values ('" +identifiant+"','"+titre+"',''"+annee+"')") 
    cursor.close()

In [ ]:
#### cacule la taille maximum 
maxa = len(df['domaine_titre'][0])
for le in df['domaine_titre'] :
    if len(str(le))>maxa:
        maxa = len(le)
maxa

21

In [ ]:
Abstract = []
for i in range(0,len(df)) :
    Abstract.append(sentense2cleanTokens(df['Abstract'][i]))
df['Abstract'] = Abstract

In [ ]:
Abstract = []
for i in range(0,len(df)) :
    Abstract.append(df['Abstract'][i].replace("β"," "))
df['Abstract'] = Abstract

In [ ]:
def t(texte) :
    lo = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    up = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    for i in str(texte):
        if ((i not in lo) and (i not in up)):
            texte = str(texte.replace(i,' '))
    return texte

e = [t(df['Abstract'][i]) for i in range(0,len(df))]

df['Abstract'] = e

In [ ]:
l = [] 
for i in range(0,len(df)):
    if len(str(df['Abstract'][i])) > 4000:
        l.append(str(df['Abstract'][i])[:4000])
    else :         
        l.append(str(df['Abstract'][i]))          
df['Abstract'] = l

In [ ]:
def sentense2cleanTokens(sent):
    sent = sent.lower()
    sent = "".join([x if x.isalpha() else " " for x in sent])
    sent = " ".join(sent.split())
    return sent

In [ ]:
titre = []
for i in range(0,len(df)) :
    titre.append(sentense2cleanTokens(df['Titre'][i]))
df['Titre'] = titre

In [ ]:
def t(texte) :
    lo = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    up = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    for i in str(texte):
        if ((i not in lo) and (i not in up)):
            texte = str(texte.replace(i,' '))
    return texte

e = [t(df['Titre'][i]) for i in range(0,len(df))]

df['Titre'] = e

In [3]:
for i in tqdm(range(3624,len(df))):
    identifiant = df['id_article'][i]
    titre = df['Titre'][i]
    annee = df['Annee'][i]
    Abstract = df['Abstract'][i]
    connection = engine.raw_connection()
    cursor = connection.cursor()
    
# Check if account exists 
    try:
        cursor = connection.cursor()
        cursor.execute("insert into Article values ('" +str(identifiant)+"','"+titre+"','"+annee+"','"+str(Abstract)+"')")
        cursor.close()
        connection.commit()
    finally:
        connection.close()

In [ ]:
df.to_csv('C:/Users/Inspiron/Desktop/Dashboard_V7.csv', index = False)

In [ ]:
################## table Keyword ##################

for i in tqdm(range(0,len(a))):
    id_keyword = i
    mot = a[i]
    connection = engine.raw_connection()
    cursor = connection.cursor()

# Check if account exists 
    try:
        cursor = connection.cursor()
        cursor.execute("insert into Keyword values (" +str(id_keyword)+",'" + str(mot)+"')")

        cursor.close()
        connection.commit()
    finally:
        connection.close()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13157/13157 [40:10<00:00,  5.52it/s]


In [ ]:
b = []

def net(mot) :
    mot = str(mot)
    return mot.lower().split('], [')[i].replace("'","").replace(",","")
def sep(mot) :
    mot = str(mot)
    return len(mot.lower().split('], ['))

for e in range(0,9000) :
    txt = df['keywords'][e]
    for i in range(0,sep(txt)):
        if i ==0:
            c = net(txt)
            b.append(c[2:])
        elif i == sep(txt)-1:
            c = net(txt)
            b.append(c[:-2])
        else:
            b.append(net(txt))
a = (list(set(b)))

In [ ]:
d = {}
def net(mot) :
    mot=str(mot)
    return mot.lower().split('], [')[i].replace("'","").replace(",","")
def sep(mot) :
    mot=str(mot)
    return len(mot.lower().split('], ['))

for e in range(23990,len(df)) :
    b = []
    txt = df['keywords'][e]
    for i in range(0,sep(txt)):
        if i ==0:
            c = net(txt)
            b.append(c[2:])
        elif i == sep(txt)-1:
            c = net(txt)
            b.append(c[:-2])
        else:
            b.append(net(txt))
    d[df['id_article'][e]] = b

In [ ]:
for identifiant in d.keys():
    liste = d[identifiant]
    for term in liste :
        position = a.index(term)
        connection = engine.raw_connection()
        cursor = connection.cursor()

        try:
            cursor = connection.cursor()
            cursor.execute("insert into Contenir values ('" +str(identifiant)+"'," + str(position)+")")

            cursor.close()
            connection.commit()
        finally:
            connection.close()

In [ ]:
mal = {}
maladie = ['cardiovascular disease','cancer','asthma' ,'alzheimer','diabetes','AIDS' ,'schizophrenia' ,'neurodegenerative diseases']
for i in range(0,len(df)):
    liste = []
    ide = df['id_article'][i]
    abstract = df['Abstract'][i]
    for e in range(0,len(maladie)) :
        if maladie[e] in str(abstract) :
            liste.append(e+1)
    mal[ide] = liste    

In [ ]:
for identifiant in mal.keys():
    liste = mal[identifiant]
    for term in liste :
        connection = engine.raw_connection()
        cursor = connection.cursor()

        try:
            cursor = connection.cursor()
            cursor.execute("insert into Concerner values ('"+str(identifiant)+"'," + str(term)+")")

            cursor.close()
            connection.commit()
        finally:s
            connection.close()

In [ ]:
###################### Auteur ########################
aut_net = []
for i in range(0,len(df)):
    auteur = df['Auteur'][i]
    aut_net.append(auteur[1:-1])

In [ ]:
auteur_net = []

for i in aut_net:
    auteur_net.append(i.replace("'", "").split(','))

In [ ]:
auteur_bis = []
for i in range(len(auteur_net)):
    trans = []
    for element in auteur_net[i]:
        if ('\\n') not in element:
            trans.append(element)
    auteur_bis.append(trans)

len(auteur_bis)

9991

In [ ]:
ab = []
for i in range(0,len(df['department'])):
    departement = re.findall('department of ([\w* ]*)',df['department'][i])
    ab.append(departement)
ab[0]

['computer science and engineering']

In [ ]:
L = ['california','massachusetts','maryland','pennsylvania','jordan',
     'texas','connecticut','ohio','carolina','minnesota','florida','michigan',
     'virginia','wisconsin','arizona','tennessee',
     'oklahoma','illinois','francisco','seattle','idaho','colorado']

etat = []
for i in tqdm(range(0,len(df))) :
    e=0
    letat = 'hors usa'
    for element in L : 
        if element in str(df['Affiliation'][i]).lower():
            letat = element
        
    etat.append(letat)


100%|████████████████████████████████████████████████████████████████████████████| 9991/9991 [00:07<00:00, 1348.45it/s]


In [ ]:
auteurs = {}
for i in tqdm(range(0,len(df))):
    a=0
    listeauteur = auteur_bis[i]
    listedepartement = ab[i]
    while (a<len(listeauteur) and a<len(listedepartement)) :
        auteurs[listeauteur[a]] = [listedepartement[a],df['Pays'][i],etat[i]]
        a+=1
    if a < len(listeauteur) :
        for e in range (a,len(listeauteur)):
            if a!=0:
                auteurs[listeauteur[e]] = [listedepartement[a-1],df['Pays'][i],etat[i]]


100%|████████████████████████████████████████████████████████████████████████████| 9991/9991 [00:01<00:00, 8948.55it/s]


In [ ]:
L = ['california','massachusetts','maryland','pennsylvania','jordan',
     'texas','connecticut','ohio','carolina','minnesota','florida','michigan',
     'virginia','wisconsin','arizona','tennessee',
     'oklahoma','illinois','francisco','seattle','idaho','colorado','states']

for element in auteurs.keys() :
    if auteurs[element][1] in L :
        auteurs[element][1] = 'usa'
    

In [ ]:
def t(texte) :
    lo = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    up = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    for i in str(texte):
        if ((i not in lo) and (i not in up)):
            texte = str(texte.replace(i,' '))
    return texte

auteursbis = {}
for cle in list(auteurs.keys()):
    l = auteurs[cle]
    cle = t(cle)
    li = []
    for element in l :
        element = t(element)
        li.append(element)
    auteursbis[cle] = li

In [ ]:
nomAuteur = list(auteursbis.keys())

for i in tqdm(range(25291,len(nomAuteur))):
    nom = nomAuteur[i]
    pays = auteursbis[nom][1]
    etat = auteursbis[nom][2]
    departement = auteursbis[nom][0]
    
    connection = engine.raw_connection()
    cursor = connection.cursor()

    try:
        cursor = connection.cursor()
        cursor.execute("insert into Auteur values ("+str(i)+",'"+str(nom)+"','" + str(pays)+"','"+str(etat)+"','"+str(departement)+"')")

        cursor.close()
        connection.commit()
    finally:
        connection.close()

In [ ]:
nom = list(auteurs.keys())
ecrire = {}
for i in tqdm(range(0,len(df))):
    url = df['id_article'][i]
    for aut in auteur_bis[i]:
        if aut in nom :
            numero = nom.index(aut)
        ecrire[str(numero)] = url

In [ ]:
for i in tqdm(range(18700,len(idauteur))):    
    identifiant = idauteur[i]
    idarticle = ecrire[identifiant]
    connection = engine.raw_connection()
    cursor = connection.cursor()
    try:
        cursor = connection.cursor()
        cursor.execute("insert into Ecrire values ('"+str(idarticle)+"',"+str(identifiant)+ ")")
        cursor.close()
        connection.commit()
    finally:
        connection.close()

In [ ]:
e = []
for i in range(0,len(df)):
    e.append(df['id_article'][i])
sup = random.sample(e,4000)

for i in tqdm(range(0,len(sup))):
    
    
    identifiant = sup[i]
    
    connection = engine.raw_connection()
    cursor = connection.cursor()
    try:
        cursor = connection.cursor()
        cursor.execute("delete from Contenir where id_article = '" +str(identifiant)+"'"  )

        cursor.close()
        connection.commit()
    finally:
        connection.close()

In [ ]:
ze = ['endoscopy','tomotherapy','gastroenterology','psychiatry','spectroscopy','mammographically','histopathology',
      'cardiology', 'oncodermatology', 'dermatology', 'neurosurgery', 'colonoscopy', 'radiology','mammography', 'photography',
 'therapy', 'neurospy', 'pathology', 'pulmonology', 'ophthalmology', 'anesthesiology', 'radiotherapy',
      'respiratory', 'biology', 'microscopy', 'urology','nephropathology','oncology','colposcopy','nanotechnology',
      'dermoscopy','morphometry','morphology','technology','chemotherapy','neuropathology']

for i in tqdm(range(0,len(ze))):
    
    
    connection = engine.raw_connection()
    cursor = connection.cursor()
    try:
        cursor = connection.cursor()
        cursor.execute("insert into DemaineApplication values(" +str(i)+",'"+ str(ze[i])+"')"  )

        cursor.close()
        connection.commit()
    finally:
        connection.close()

In [ ]:
liste = []
for i in range(0,len(df)):
    abstract = str(df['Abstract'][i])
    abstract = abstract.replace('using','').replace('used','').replace('based','').replace('feature','')
    liste.append(abstract)
df['Abstract'] = liste

In [ ]:
##########DOMAINES Application ###############
list_spe = ['drug design','detection','patient care']

DictDomaine = {}
for i in range(0,len(df)) :
    identifiant = df['id_article'][i]
    abstract = df['Abstract'][i]
    titre = df['Titre'][i]
    liste = []
    for element in list_spe :
        if (element in str(abstract)) or (element in str(titre)):
            liste.append(element)
    DictDomaine[identifiant] = liste

Domaines = {}
for i in range (0,len(list_spe)):
    dom = list_spe[i]
    Domaines[dom] = str(i) 

In [ ]:
for cle,valeur in Domaines.items():
    connection = engine.raw_connection()
    cursor = connection.cursor()
    try:
        cursor = connection.cursor()
        cursor.execute("insert into DomaineApplication values(" +str(valeur)+",'"+ str(cle)+"')"  )

        cursor.close()
        connection.commit()
    finally:
        connection.close()
    

In [ ]:
############### Traiter ###############
for i in tqdm(range(0,len(df))):
    identifiant = df['id_article'][i]
    liste = DictDomaine[identifiant]
    for domaine in liste :
        
        connection = engine.raw_connection()
        cursor = connection.cursor()
        try:
            cursor = connection.cursor()
            cursor.execute("insert into Traiter values('" +str(identifiant)+"',"+ str(Domaines[domaine])+")"  )

            cursor.close()
            connection.commit()
        finally:
            connection.close()

100%|██████████████████████████████████████████████████████████████████████████████| 9991/9991 [14:58<00:00,  6.85it/s]


In [ ]:
############### METHODE APPRENTISSAGE ###############
list_spe = ['knn','neural network','logistic regression','decision tree','k means','machine learning','deep learning','robot']

DictDomaine = {}
for i in range(0,len(df)) :
    identifiant = df['id_article'][i]
    abstract = df['Abstract'][i]
    titre = df['Titre'][i]
    liste = []
    for element in list_spe :
        if (element in str(abstract)) or (element in str(titre)):
            liste.append(element)
    DictDomaine[identifiant] = liste

Domaines = {}
for i in range (0,len(list_spe)):
    dom = list_spe[i]
    Domaines[dom] = str(i) 

In [ ]:
############### MethodeApprentissage ###############
for cle,valeur in Domaines.items():
    connection = engine.raw_connection()
    cursor = connection.cursor()
    try:
        cursor = connection.cursor()
        cursor.execute("insert into MethodeApprentissage values(" +str(valeur)+",'"+ str(cle)+"')"  )

        cursor.close()
        connection.commit()
    finally:
        connection.close()

In [ ]:
############### Parler ###############
for i in tqdm(range(0,len(df))):
    identifiant = df['id_article'][i]
    liste = DictDomaine[identifiant]
    for domaine in liste :
        
        connection = engine.raw_connection()
        cursor = connection.cursor()
        try:
            cursor = connection.cursor()
            cursor.execute("insert into Parler values('" +str(identifiant)+"',"+ str(Domaines[domaine])+")"  )

            cursor.close()
            connection.commit()
        finally:
            connection.close()


100%|██████████████████████████████████████████████████████████████████████████████| 9991/9991 [18:33<00:00,  4.79it/s]


In [ ]:
connection = engine.raw_connection()
cursor = connection.cursor()
try:
    cursor = connection.cursor()
    cursor.execute("select count(*) from article")
    
    for ligne in cursor:
        print(ligne)
    cursor.close()
    connection.commit()
finally:
    connection.close()


In [ ]:
id_article = []
titre = []
date = []
abstract = []

connection = engine.raw_connection()
cursor = connection.cursor()
try:
    cursor = connection.cursor()
    cursor.execute("select * from article")

    for ligne in cursor:
        id_article.append(ligne[0])
        titre.append(ligne[1])
        date.append(ligne[2])
        abstract.append(ligne[3])
    cursor.close()
    connection.commit()
finally:
    connection.close()


In [ ]:
Dataframe['id_article']=id_article
Dataframe['Titre']=titre
Dataframe['Abstract']=abstract

In [ ]:
Pays={}
connection = engine.raw_connection()
cursor = connection.cursor()
try:
    cursor = connection.cursor()
    cursor.execute("select pays,id_article from auteur,ecrire where auteur.id_auteur=ecrire.id_auteur")
    
    for ligne in cursor:
        Pays[ligne[1]]=ligne[0]
    cursor.close()
    connection.commit()
finally:
    connection.close()

In [ ]:
len(Pays)

6221

In [ ]:
liste = []
for i in range(0,len(df)) :
    id_article = df['id_article'][i]
    if id_article in Pays.keys():
        liste.append(Pays[id_article])
    else:
        liste.append([])

In [ ]:
Dataframe['Pays']=liste

In [ ]:
keyword={}
liste=[]
connection = engine.raw_connection()
cursor = connection.cursor()
try:
    cursor = connection.cursor()
    cursor.execute("select c.id_article,k.mot from keyword k,contenir c where k.id_keyword=c.id_keyword")
    
    for ligne in cursor:
        if ligne[0] in keyword.keys():
            keyword[ligne[0]] = keyword[ligne[0]] + [ligne[1]]
        else :
            keyword[ligne[0]] = [ligne[1]]
    cursor.close()
    connection.commit()
finally:
    connection.close()

In [ ]:
len(keyword)

3806

In [ ]:
liste = []
for i in range(0,len(df)) :
    id_article = df['id_article'][i]
    if id_article in keyword.keys():
        liste.append(keyword[id_article])
    else:
        liste.append([])

In [ ]:
Dataframe['keyword']=liste

In [ ]:
listeid = []
connection = engine.raw_connection()
cursor = connection.cursor()
try:
    cursor = connection.cursor()
    cursor.execute("select distinct(id_article) from contenir"  )
    for element in cursor :
        listeid.append(element[0])
    cursor.close()
    connection.commit()
finally:
    connection.close()

In [ ]:
############### Ecrire ###############
idauteur = list(ecrire.keys())
for i in tqdm(range(0,len(idauteur))):
    
    identifiant = idauteur[i]
    idarticle = ecrire[identifiant]
    if idarticle in listeid :
        connection = engine.raw_connection()
        cursor = connection.cursor()
        try:
            cursor = connection.cursor()
            cursor.execute("insert into Ecrire values ('"+str(idarticle)+"',"+str(idauteur)+ ")")

            cursor.close()
            connection.commit()
        finally:
            connection.close()